## HuReTEx GA 0.01 (2025.03.13)

In [4]:
import numpy as np
import pandas as pd
from geneticalgorithm import geneticalgorithm as ga

In [ ]:
def fitness(x, rsfg_layers):

    confidences = list()

    for i in range(len(x)-1):

        layer = rsfg_layers[i]

        confidence = layer.loc[(layer['source_id']==x[i]) & (layer['target_id']==x[i+1]), ['confidence']]

        if len(confidence)==0:

            confidences.append(0.0)

        else:  
            
            confidences.append(confidence['confidence'])

    assessment = np.prod(np.array(confidences)) 

    if assessment == 0.0:
        assessment = assessment + 0.0000000001

    return -assessment  

In [ ]:
def get_best_confident_path(rsfg_layers, node_dicts, n_clusters_conv_1, varbounds):
    
    max_num_iteration = 10000

    parameters = {'max_num_iteration': max_num_iteration,
              'population_size': 20,
              'parents_portion': 0.3,
              'mutation_probability': 0.1,
              'crossover_probability': 0.1,
              'elit_ratio': 0.01,
              'crossover_type': 'uniform',
              'max_iteration_without_improv': int(0.2*max_num_iteration)}


    gen_alg = ga(function=lambda x: fitness(x, rsfg_layers), dimension=len(varbounds), variable_type='int', variable_boundaries=np.array(varbounds), algorithm_parameters=parameters)

    gen_alg.run()

    best_path = gen_alg.output_dict['variable']

    best_path_info = pd.DataFrame(columns=['confidence', 'source', 'target'])

    for i in range(len(best_path)-1):

            layer = rsfg_layers[i]

            path_info = layer.loc[(layer['source_id']==best_path[i]) & (layer['target_id']==best_path[i+1]), ['confidence', 'source', 'target']]
        
            best_path_info = pd.concat([best_path_info.dropna(axis=1, how='all') , path_info], ignore_index=True)

    best_solution = -gen_alg.output_dict['function']

    return (best_path_info, best_solution)